In [29]:
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [30]:
dir = 'C:/Users/alan/Medical Image Project'
# Load your pre-trained models
cnn_model = load_model(dir +'/CNN_model.h5')
nlp_model = load_model(dir +'/NLP_model.h5')

In [31]:
text_data = dir + "\\BrEaST-Lesions-USG-clinical-data-Dec-15-2023.xlsx"

In [32]:
df = pd.read_excel(text_data)

In [33]:
# Preprocessing
columns_to_drop = ["Image_filename", "Mask_tumor_filename","Mask_other_filename", "Pixel_size", "Verification","BIRADS"]
new_df = df.drop(columns=columns_to_drop)


# handling missing Age
new_df_filtered = new_df[new_df['Age'] != 'not available']
new_df_filtered['Age'] = pd.to_numeric(new_df_filtered['Age'])
age_mean_corrected = new_df_filtered['Age'].mean()

# Replace 'not available' with the calculated mean age in the original new_df
new_df['Age'] = new_df['Age'].replace('not available', age_mean_corrected)
new_df['Age'] = pd.to_numeric(new_df['Age'], errors='coerce')
new_df['Age'] = new_df['Age'].astype(int)

# Drop rows where the 'Classification' column does not contain 'benign' or 'malignant'
final_df = new_df[(new_df['Classification'] == 'benign') | (new_df['Classification'] == 'malignant')]

label = final_df['Classification']
final_df = final_df.drop('Classification', axis=1)

C:\Users\alan\AppData\Local\Temp\ipykernel_14076\2879118811.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_filtered['Age'] = pd.to_numeric(new_df_filtered['Age'])


In [34]:
# NLP feature extraction

non_numerical_columns = [col for col in final_df.columns if final_df[col].dtype == 'object' and col not in ['CaseID', 'Age']]
# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=100, stop_words='english')

for column in non_numerical_columns:
    # Convert column to string
    column_as_string = final_df[column].astype(str).fillna('')
    tfidf_matrix = tfidf_vectorizer.fit_transform(column_as_string)
    
    feature_names = [f"{column}_{word}" for word in tfidf_vectorizer.get_feature_names_out()]
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
    
    # Concatenate TF-IDF 
    if 'final_numerical_df' in locals():
        final_numerical_df = pd.concat([final_numerical_df, tfidf_df], axis=1)
    else:
        final_numerical_df = tfidf_df.copy()

        
final_numerical_df = pd.concat([final_df[['CaseID', 'Age']].reset_index(drop=True), final_numerical_df.reset_index(drop=True)], axis=1)

In [35]:
# split the data
X = final_numerical_df

encoder = LabelEncoder()
y = encoder.fit_transform(label)

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((201, 312), (51, 312), (201,), (51,))

In [38]:
text_predictions = nlp_model.predict(X_test)


ValueError: in user code:

    File "G:\anaconda3\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "G:\anaconda3\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "G:\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "G:\anaconda3\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "G:\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "G:\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequential_28" is incompatible with the layer: expected s

In [ ]:
s